# Loop API Example on Hartmann6

Ax' managed loop API is meant to allow for reduction of optimization to a single call and for CLI compatibility.

In [14]:
import numpy as np
from ax.service.managed_loop import OptimizationLoop, OptimizationPlan, optimize
from ax.metrics.branin import branin

In [ ]:
def hartmann6(x: np.ndarray) -> float:
        alpha = np.array([1.0, 1.2, 3.0, 3.2])
        A = np.array(
            [
                [10, 3, 17, 3.5, 1.7, 8],
                [0.05, 10, 17, 0.1, 8, 14],
                [3, 3.5, 1.7, 10, 17, 8],
                [17, 8, 0.05, 10, 0.1, 14],
            ]
        )
        P = 10 ** (-4) * np.array(
            [
                [1312, 1696, 5569, 124, 8283, 5886],
                [2329, 4135, 8307, 3736, 1004, 9991],
                [2348, 1451, 3522, 2883, 3047, 6650],
                [4047, 8828, 8732, 5743, 1091, 381],
            ]
        )
        y = 0.0
        for j, alpha_j in enumerate(alpha):
            t = 0
            for k in range(6):
                t += A[j, k] * ((x[k] - P[j, k]) ** 2)
            y -= alpha_j * np.exp(-t)
        return y

First, we set up an evaluation function that returns evaluations for two metrics: "hartmann6" and "l2norm".

In [16]:
def hartmann_evaluation_function(
    parameterization, # dict of parameter names to values of those parameters
    weight=None, # evaluation function signature requires a weight argument
):
    x = np.array([parameterization.get(f"x{i+1}") for i in range(6)])
    # In our case, standard error is 0, since we are computing a synthetic function.
    return {"hartmann6": (hartmann6(x), 0.0), "l2norm": (np.sqrt((x ** 2).sum()), 0.0)}

The setup for the loop is similar to that of the service API and will be fully compatible with JSON. Currently, the optimization algorithm is chosen under the hood based on the properties of the problem search space, but it will be exposed for choice shortly.

In [17]:
optimize(
    parameters=[
        {
            "name": "x1",
            "type": "range",
            "bounds": [0.0, 1.0],
            "value_type": "float",  # Optional, defaults to inference from type of "bounds".
            "log_scale": False,  # Optional, defaults to False.
        },
        {
            "name": "x2",
            "type": "range",
            "bounds": [0.0, 1.0],
        },
        {
            "name": "x3",
            "type": "range",
            "bounds": [0.0, 1.0],
        },
        {
            "name": "x4",
            "type": "range",
            "bounds": [0.0, 1.0],
        },
        {
            "name": "x5",
            "type": "range",
            "bounds": [0.0, 1.0],
        },
        {
            "name": "x6",
            "type": "range",
            "bounds": [0.0, 1.0],
        },
    ],
    experiment_name="test",
    objective_name="hartmann6",
    evaluation_function=hartmann_evaluation_function,
    minimize=True,  # Optional, defaults to False.
    parameter_constraints=["x1 + x2 <= 20"],  # Optional.
    outcome_constraints=["l2norm <= 1.25"],  # Optional.
    optimization_plan=OptimizationPlan(total_iterations=10),  # Optional.
)

[INFO 04-17 15:19:35] ax.service.utils.dispatch: Using Bayesian Optimization generation strategy. Iterations after 6 will take longer to generate due to model-fitting.
[INFO 04-17 15:19:35] ax.service.managed_loop: Started full optimization with 10 steps.
[INFO 04-17 15:19:35] ax.service.managed_loop: Running optimization step 1...
[INFO 04-17 15:19:35] ax.service.managed_loop: Running optimization step 2...
[INFO 04-17 15:19:35] ax.service.managed_loop: Running optimization step 3...
[INFO 04-17 15:19:35] ax.service.managed_loop: Running optimization step 4...
[INFO 04-17 15:19:35] ax.service.managed_loop: Running optimization step 5...
[INFO 04-17 15:19:35] ax.service.managed_loop: Running optimization step 6...
[INFO 04-17 15:19:35] ax.service.managed_loop: Running optimization step 7...
[WARNING 04-17 15:19:35] IVW: Conflicting noiseless measurements for hartmann6.
[WARNING 04-17 15:19:35] IVW: Conflicting noiseless measurements for l2norm.
[INFO 04-17 15:23:47] ax.service.managed_

{'x1': 0.25260233609096855,
 'x2': 0.18264137184291412,
 'x3': 0.326031849086933,
 'x4': 0.16036655372973474,
 'x5': 0.5867112805724518,
 'x6': 0.511645052865211}